In [1]:
#!pip3 install dowhy --user 

## Imports

In [2]:
import dowhy
import numpy as np
import pandas as pd
from dowhy import CausalModel
import time

In [3]:
import warnings
import logging

In [4]:
matching_data = "../../matching_data/data.p"
data = pd.read_pickle(matching_data)

## Some Processing

In [5]:
#drop person_id
data = data.drop(columns=["PERS_ID"],axis=1)
data.loc[:,"BIOMETRIC_CATEGORY"] = 0

In [6]:
#generate X
X = pd.concat([data.loc[:,:"MBR_GNDR"], data.loc[:,"RESULT_VALUE"]], axis=1)

#generate Y
Y = data.OUTCOME2

#generate T
T = data.TREATED

In [7]:
data = pd.concat([X,Y,T], axis=1)
data = data.astype({"TREATED":'bool'}, copy=False)

In [8]:
data.head()

AGE_AT_MIDMONTH  ALLOW_AMT  BIOMETRIC_CATEGORY  CC_ADHD  CC_ALZHEIMER  \
0               81        414                   0        0             0   
1               82        101                   0        0             0   
2               78         72                   0        0             0   
3               78      13007                   0        0             0   
4               76        898                   0        0             0   

   CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  \
0                  0       0          0                    1             0   
1                  0       0          0                    1             0   
2                  0       0          0                    0             0   
3                  0       0          0                    0             1   
4                  0       0          0                    0             1   

   ...  CC_SPINAL_CORD  CC_SPINA_BIFIDA  CC_STROKE  CC_TOBACCO  \
0  ...               0                0          0           0   
1  ...               0                0          0           0   
2  ...               0                0          0           0   
3  ...               0                0          0           0   
4  ...               0                0          0           0   

   CC_TRAUMATIC_BRAIN  CC_VISUAL_IMPAIR  MBR_GNDR  RESULT_VALUE  OUTCOME2  \
0                   0                 0         0     27.772373  0.007732   
1                   0                 0         0     29.256837  0.007214   
2                   0                 0         1     29.575874 -0.058446   
3                   0                 0         1     23.289228  0.099021   
4                   0                 0         1     40.305640 -0.340599   

   TREATED  
0     True  
1     True  
2     True  
3     True  
4     True  

[5 rows x 76 columns]

## Apply Do Why - 

In [9]:
common_causes = X.columns.tolist()

In [12]:
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)
warnings.filterwarnings("ignore")

### 1. Propensty Score Matching

In [13]:
%%time
#----------------------------
#step 1: Create Causal Model
#----------------------------
PSM_model=CausalModel(
        data = data,
        treatment='TREATED',
        outcome='OUTCOME2',
        common_causes=common_causes
        )
print("Step 1: Causal Model Created")
#------------------------------
#step 2: Identify the Estimand
#------------------------------
PSM_identified_estimand = PSM_model.identify_effect(proceed_when_unidentifiable=True)
print("Step 2: Estimand Identified")

#------------------------------
#step 3: Estimating the Effect
#------------------------------
PSM_estimate = PSM_model.estimate_effect(PSM_identified_estimand,
        method_name="backdoor.propensity_score_matching",
        target_units = "ate"
)
print("Step 3: Estimated the treatment effect")
print("Causal Estimate is " + str(PSM_estimate.value))

#------------------------------
#step 4: Refutation
#------------------------------
print("Step 4: Refutation")
#refute using common cause
#solution: Does the estimation method change its estimate after we add an independent random variable as a common cause to the dataset? (Hint: It should not)
refute_random_common_cause = PSM_model.refute_estimate(
                                        PSM_identified_estimand,
                                        PSM_estimate,
                                        method_name="random_common_cause"
                                    )
print("a. After random common cause refutation --")
print(refute_random_common_cause)

#refute using placebo treatment
#What happens to the estimated causal effect when we replace the true treatment variable with an independent random variable? (Hint: the effect should go to zero)
refute_placebo_treatment = PSM_model.refute_estimate(
                                    PSM_identified_estimand,
                                    PSM_estimate,
                                    method_name="placebo_treatment_refuter",
                                    placebo_type="permute"
                                )
print("b. After placebo treatment refutation --")
print(refute_placebo_treatment)

#refute using random subset
#Does the estimated effect change significantly when we replace the given dataset with a randomly selected subset? (Hint: It should not)
refute_random_subset = PSM_model.refute_estimate(
                                    PSM_identified_estimand,
                                    PSM_estimate,
                                    method_name="data_subset_refuter",
                                    subset_fraction=0.8
                            )
print("c. After random subset refutation --")
print(refute_random_subset)

Step 1: Causal Model Created
Step 2: Estimand Identified
Step 3: Estimated the treatment effect
Causal Estimate is -0.013654600197429468
Step 4: Refutation
a. After random common cause refutation --
Refute: Add a Random Common Cause
Estimated effect:(-0.013654600197429468,)
New effect:(-0.007595663384530591,)

b. After placebo treatment refutation --
Refute: Use a Placebo Treatment
Estimated effect:(-0.013654600197429468,)
New effect:(-0.003013367692865043,)

c. After random subset refutation --
Refute: Use a subset of data
Estimated effect:(-0.013654600197429468,)
New effect:(-0.01757214772880864,)

CPU times: user 6min 30s, sys: 1min 32s, total: 8min 3s
Wall time: 4min 21s


### 2. Propensty Score Stratification

In [14]:
%%time
#----------------------------
#step 1: Create Causal Model
#----------------------------
PSS_model=CausalModel(
        data = data,
        treatment='TREATED',
        outcome='OUTCOME2',
        common_causes=common_causes
        )
print("Step 1: Causal Model Created")

#------------------------------
#step 2: Identify the Estimand
#------------------------------
PSS_identified_estimand = PSS_model.identify_effect(proceed_when_unidentifiable=True)
print("Step 2: Estimand Identified")

#------------------------------
#step 3: Estimating the Effect
#------------------------------
PSS_estimate = PSS_model.estimate_effect(PSS_identified_estimand,
                                              method_name="backdoor.propensity_score_stratification",
                                              target_units="ate")
print("Step 3: Estimated the treatment effect")
print("Causal Estimate is " + str(PSS_estimate.value))

#------------------------------
#step 4: Refutation
#------------------------------
print("Step 4: Refutation")
#refute using common cause
#solution: Does the estimation method change its estimate after we add an independent random variable as a common cause to the dataset? (Hint: It should not)
refute_random_common_cause = PSS_model.refute_estimate(
                                        PSS_identified_estimand,
                                        PSS_estimate,
                                        method_name="random_common_cause"
                                    )
print("a. After random common cause refutation --")
print(refute_random_common_cause)

#refute using placebo treatment
#What happens to the estimated causal effect when we replace the true treatment variable with an independent random variable? (Hint: the effect should go to zero)
refute_placebo_treatment = PSS_model.refute_estimate(
                                    PSS_identified_estimand,
                                    PSS_estimate,
                                    method_name="placebo_treatment_refuter",
                                    placebo_type="permute"
                                )
print("b. After placebo treatment refutation --")
print(refute_placebo_treatment)

#refute using random subset
#Does the estimated effect change significantly when we replace the given dataset with a randomly selected subset? (Hint: It should not)
refute_random_subset = PSS_model.refute_estimate(
                                    PSS_identified_estimand,
                                    PSS_estimate,
                                    method_name="data_subset_refuter",
                                    subset_fraction=0.8
                            )
print("c. After random subset refutation --")
print(refute_random_subset)

Step 1: Causal Model Created
Step 2: Estimand Identified
Step 3: Estimated the treatment effect
Causal Estimate is 0.004090213777337744
Step 4: Refutation
a. After random common cause refutation --
Refute: Add a Random Common Cause
Estimated effect:(0.004090213777337744,)
New effect:(0.00641786891885523,)

b. After placebo treatment refutation --
Refute: Use a Placebo Treatment
Estimated effect:(0.004090213777337744,)
New effect:(-0.006783792442400828,)

c. After random subset refutation --
Refute: Use a subset of data
Estimated effect:(0.004090213777337744,)
New effect:(0.004905582428751893,)

CPU times: user 9.92 s, sys: 4.16 s, total: 14.1 s
Wall time: 8.12 s


### 3. Propensty Score Weighting

In [15]:
%%time
#----------------------------
#step 1: Create Causal Model
#----------------------------
PSW_model=CausalModel(
        data = data,
        treatment='TREATED',
        outcome='OUTCOME2',
        common_causes=common_causes
        )
print("Step 1: Causal Model Created")

#------------------------------
#step 2: Identify the Estimand
#------------------------------
PSW_identified_estimand = PSW_model.identify_effect(proceed_when_unidentifiable=True)
print("Step 2: Estimand Identified")

#------------------------------
#step 3: Estimating the Effect
#------------------------------
PSW_estimate = PSW_model.estimate_effect(PSW_identified_estimand,
                                            method_name="backdoor.propensity_score_weighting",
                                            target_units = "ate",
                                            method_params={"weighting_scheme":"ips_weight"})
print("Step 3: Estimated the treatment effect")
print("Causal Estimate is " + str(PSW_estimate.value))

#------------------------------
#step 4: Refutation
#------------------------------
print("Step 4: Refutation")
#refute using common cause
#solution: Does the estimation method change its estimate after we add an independent random variable as a common cause to the dataset? (Hint: It should not)
refute_random_common_cause = PSW_model.refute_estimate(
                                        PSW_identified_estimand,
                                        PSW_estimate,
                                        method_name="random_common_cause"
                                    )
print("a. After random common cause refutation --")
print(refute_random_common_cause)

#refute using placebo treatment
#What happens to the estimated causal effect when we replace the true treatment variable with an independent random variable? (Hint: the effect should go to zero)
refute_placebo_treatment = PSW_model.refute_estimate(
                                    PSW_identified_estimand,
                                    PSW_estimate,
                                    method_name="placebo_treatment_refuter",
                                    placebo_type="permute"
                                )
print("b. After placebo treatment refutation --")
print(refute_placebo_treatment)

#refute using random subset
#Does the estimated effect change significantly when we replace the given dataset with a randomly selected subset? (Hint: It should not)
refute_random_subset = PSW_model.refute_estimate(
                                    PSW_identified_estimand,
                                    PSW_estimate,
                                    method_name="data_subset_refuter",
                                    subset_fraction=0.8
                           )
print(refute_random_subset)

Step 1: Causal Model Created
Step 2: Estimand Identified
Step 3: Estimated the treatment effect
Causal Estimate is -0.001279416760332313
Step 4: Refutation
a. After random common cause refutation --
Refute: Add a Random Common Cause
Estimated effect:(-0.001279416760332313,)
New effect:(-0.0012794059987848287,)

b. After placebo treatment refutation --
Refute: Use a Placebo Treatment
Estimated effect:(-0.001279416760332313,)
New effect:(-0.0019108355996638629,)

Refute: Use a subset of data
Estimated effect:(-0.001279416760332313,)
New effect:(-0.0014919576918384532,)

CPU times: user 9.3 s, sys: 4.1 s, total: 13.4 s
Wall time: 7.37 s
